# Tutorial 05: weak imposition of Dirichlet BCs by a Lagrange multiplier (inf-sup computation)

In this tutorial we compute the inf-sup constant of the saddle point problem resulting from the discretization of the following Laplace problem
$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = 0, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D, and for which the non-homogeneous Dirichlet boundary conditions are imposed by a Lagrange multiplier.

The resulting eigenvalue problem is
$$
\text{find } \eta, u, \lambda \in \mathbb{R} \times V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla u \cdot \nabla v + \int_\Gamma \lambda v = 0, & \forall v \in V,\\
\int_\Gamma u \mu = \eta \int_\Gamma \lambda \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc
from ufl import dx, grad, inner, Measure, TestFunction, TrialFunction
from dolfinx import Constant, FunctionSpace
from dolfinx.fem import locate_dofs_topological
from dolfinx.io import XDMFFile
from multiphenicsx.fem import assemble_matrix_block, DofMapRestriction

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.topology.create_connectivity_all()
    boundaries = infile.read_meshtags(mesh, name="boundaries")
facets_Gamma = boundaries.indices[boundaries.values == 1]

In [ ]:
# Define associated measures
ds = Measure("ds")(subdomain_data=boundaries)

### Eigenvalue problem

In [ ]:
# Define a function space
V = FunctionSpace(mesh, ("Lagrange", 2))

In [ ]:
# Define restrictions.
dofs_V = np.arange(0, V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
restriction_V = DofMapRestriction(V.dofmap, dofs_V)
restriction_V_Gamma = DofMapRestriction(V.dofmap, dofs_V_Gamma)
restriction = [restriction_V, restriction_V_Gamma]

In [ ]:
# Define trial and test functions
(u, l) = (TrialFunction(V), TrialFunction(V))
(v, m) = (TestFunction(V), TestFunction(V))

In [ ]:
# Define problem block forms
a = [[inner(grad(u), grad(v)) * dx, - l * v * ds],
     [- u * m * ds, None]]
b = [[None, None],
     [None, - l * m * ds]]
b[0][0] = Constant(mesh, 0.) * inner(u, v) * dx

In [ ]:
# Assemble lhs and rhs matrices
A = assemble_matrix_block(a, bcs=[], restriction=(restriction, restriction))
A.assemble()
B = assemble_matrix_block(b, bcs=[], restriction=(restriction, restriction))
B.assemble()

In [ ]:
# Solve
eps = SLEPc.EPS().create()
eps.setOperators(A, B)
eps.setProblemType(SLEPc.EPS.ProblemType.GNHEP)
eps.setDimensions(1, PETSc.DECIDE, PETSc.DECIDE)
eps.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
eps.setTarget(1.e-5)
eps.getST().setType(SLEPc.ST.Type.SINVERT)
eps.getST().getKSP().setType("preonly")
eps.getST().getKSP().getPC().setType("lu")
eps.getST().getKSP().getPC().setFactorSolverType("mumps")
eps.solve()
assert eps.getConverged() >= 1

In [ ]:
# Extract leading eigenvalue
eigv = eps.getEigenvalue(0)
r, i = eigv.real, eigv.imag
assert abs(i) < 1.e-10
assert r > 0., "r = " + str(r) + " is not positive"
print("Inf-sup constant: ", np.sqrt(r))
assert np.isclose(np.sqrt(r), 0.125496)